In [9]:
"""
Created on Tue Oct  4 22:20:13 2016
@author: Sarick Shah
"""
'''
Update to python3
:author: Kaleb Nunn
'''

import numpy as np
import urllib3 as urllib2
from bs4 import BeautifulSoup
from collections import defaultdict 
from pandas import Series, DataFrame
import re, sys, unidecode, unicodedata

sys.setrecursionlimit(20000)

In [10]:
def solver(list_of_variables, mpaaRating): 
    list_to_append = []
    for category in list_of_variables: 
        try: 
            if category == 'Rotten Tomatoes':  
                rating_index = mpaaRating.index(category)+3
                rating = mpaaRating[rating_index:rating_index+2]                
                list_to_append.extend([rating])                
            else:
                category = (mpaaRating[mpaaRating.index(category)+1])
                list_to_append.append(category)
                
        except (ValueError, AttributeError):
            #print "category didn't work"
            list_to_append.append(None)            
    return list_to_append

In [12]:
http = urllib3.PoolManager()

In [25]:
response = http.request('GET', 'http://www.the-numbers.com/movie/budgets/all')
main_doc = response.data
main_doc

b'<!DOCTYPE html>\r\n<html xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml">\r\n<head>\r\n<meta http-equiv="PICS-Label" content=\'(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/ratingsv01.html" l gen true for "https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))\'>\r\n<!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n<meta name="format-detection" content="telephone=no">   <!-- for apple mobile --> \r\n<meta property="fb:admins" content="521546213" />\r\n\r\n<meta name="viewport" content="initial-scale=1">\n<meta name="description" content="The budget of (nearly) every movie ever made">\n<meta name="robots" content="NOODP">\n<meta name="keywords" content="movies, box office, The Numbers, Numbers, daily box office, weekly box office, movie stars, dvd sales, Blu-ray

In [38]:
def txt_link_downloader(html_link):
    
    soup = BeautifulSoup(html_link, 'html.parser')
    list_df = []    
    batch = soup.find_all('td')
    counter = 0
    for index,i in enumerate(range(0,len(batch),6)):
        list_df.append(map(lambda x: x.get_text(), batch[i:i+6]))
        
        url_end = BeautifulSoup(batch[i+2].encode('utf-8'),'html.parser').find('a').get('href') 
        url = 'http://www.the-numbers.com' + url_end
        #list_df[index].append(url)
        
        http = urllib3.PoolManager()
        response = http.request('GET', url)
        main_doc = response.data
        soup = BeautifulSoup(main_doc,'html.parser')
    
        mpaaRating = []
        for tr in soup.findAll('tr'): 
            for td in tr.findAll('td'): 
                mpaaRating.append(td.get_text())
        mpaaRating = [unidecode.unidecode(x).strip() for x in mpaaRating]   
        
        list_of_variables = ['Genre:','Running Time:','MPAA Rating:','Production Companies:','Domestic Releases:','Domestic DVD Sales','Domestic Blu-ray Sales','Total Domestic Video Sales','Rotten Tomatoes']
        
        second_page = solver(list_of_variables,mpaaRating)
        list_df.append(second_page)
        
        http = urllib3.PoolManager()
        response = http.request('GET', url)
        main_doc = response.data
        soup = BeautifulSoup(main_doc,'html.parser')
        soup = soup.find(text = re.compile('Weekend Box Office Performance')).parent.parent.find('div', attrs = {"id": "box_office_chart"})
        try:
            soup = soup.get_text()
            soup = unicodedata.normalize('NFKD', soup).encode('utf-8').split()[4:35]
            soup.insert(3,'None')
            list_df.append(soup)
        except:
            pass
        
        counter += 1
        #sets upper limit, max is 5230 as of 10/9/2016
        if counter == 2000:
            return DataFrame(list_df)            

In [39]:
import copy
list_df = txt_link_downloader(main_doc)

df_deepcopy = copy.deepcopy(list_df)
display(df_deepcopy)

[<td class="data">1</td>, <td><a href="/box-office-chart/daily/2019/04/23">Apr 23, 2019</a></td>, <td><b><a href="/movie/Avengers-Endgame-(2019)#tab=summary">Avengers: Endgame</a></b></td>, <td class="data"> $400,000,000</td>, <td class="data"> $858,373,000</td>, <td class="data"> $2,797,800,564</td>, <td class="data">2</td>, <td><a href="/box-office-chart/daily/2011/05/20">May 20, 2011</a></td>, <td><b><a href="/movie/Pirates-of-the-Caribbean-On-Stranger-Tides#tab=summary">Pirates of the Caribbean: On Stranger Tides</a></b></td>, <td class="data"> $379,000,000</td>, <td class="data"> $241,063,875</td>, <td class="data"> $1,045,663,875</td>, <td class="data">3</td>, <td><a href="/box-office-chart/daily/2015/04/22">Apr 22, 2015</a></td>, <td><b><a href="/movie/Avengers-Age-of-Ultron#tab=summary">Avengers: Age of Ultron</a></b></td>, <td class="data"> $365,000,000</td>, <td class="data"> $459,005,868</td>, <td class="data"> $1,396,099,202</td>, <td class="data">4</td>, <td><a href="/box-

None

In [40]:
df_deepcopy.to_csv('sarickmovies3.csv', encoding= 'utf-8' )

AttributeError: 'NoneType' object has no attribute 'to_csv'